<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/02_intermediate/Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

tf.enable_eager_execution()
L = tf.keras.layers
tfe = tf.contrib.eager

In [3]:
# Hyper parameters
num_epochs = 25
num_classes = 10
batch_size = 512
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("training_data: ", x_train.shape)
print("test_data: ", x_test.shape)
print("training_label: ", y_train.shape)
print("test_label: ", y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000,)
test_label:  (10000,)


In [4]:
x_train_eager = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test_eager = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_train_eager = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))
y_test_eager = tf.reshape(tf.one_hot(y_test, 10), (-1, 10))

print("training_data: ", x_train_eager.shape)
print("test_data: ", x_test_eager.shape)
print("training_label: ", y_train_eager.shape)
print("test_label: ", y_test_eager.shape)

training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000, 10)
test_label:  (10000, 10)


### DataSet
You make Dataset using `tf.data.Dataset` Class but Keras API doesn't need this dataset. If you write training loop code manually, `Dataset` class is very useful. And using keras API, you need numpy.array inputs instead of tf.Tensor. I don't know why...so you only need numpy preprocessing (or get numpy.array from tf.Tensor using numpy() method after preprocessing using function of tf).

### NOTE
This notebook we don't need 'tf.data.Dataset'. This code only just for reference.

In [0]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train_eager, y_train_eager))
    .batch(batch_size)
    .shuffle(10000)
)
train_dataset = train_dataset.repeat()

In [0]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test_eager, y_test_eager))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = test_dataset.repeat()

### RNN using LSTM
In keras API, LSTM recives inputs tensor whose shape is (batch_size, seqence_length, feature_dim), and output tensor whose shape is (batch_size, fearure_dim).When you need all time sequence data, you have to give `return_sequences=True` to LSTM's constractor. Generally, when you stack LSTM's, you need all sequence data.

In [0]:
class RNN(tf.keras.Model):
    def __init__(self, hidden_size=10, num_layers=2, num_classes=10):
        super(RNN, self).__init__(name='mnist_rnn')
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = self.get_lstm_layers(hidden_size, num_layers)            
        self.fc = L.Dense(num_classes, activation="softmax")
    
    @staticmethod
    def get_lstm_layers(hidden_size, num_layers):
        lstm_layers = []
        # we need all sequence data. write return_sequences=True! 
        for i in range(num_layers-1):
            lstm_layers.append(
                L.CuDNNLSTM(units=hidden_size, return_sequences=True)
            )
        # the final layer return only final sequence
        # if you need all sequences, you have to write return_sequences=True.
        lstm_layers.append(L.CuDNNLSTM(units=hidden_size))
        return tf.keras.Sequential(lstm_layers)
        
    def call(self, x):        
        # Forward propagate LSTM
        out = self.lstm(x)
        out = self.fc(out)
        return out

In [0]:
model = RNN()

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=["accuracy"])

# Eager Execution initialize parameters when using model.call()
model(x_train_eager[:50])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  2480      
_________________________________________________________________
dense (Dense)                multiple                  110       
Total params: 2,590
Trainable params: 2,590
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(x=x_train_eager.numpy(), 
          y=y_train_eager.numpy(), 
          validation_split=0.2, 
          epochs=num_epochs,
          batch_size=batch_size)

Epoch 1/25
94/94 [==============================] - 5s 50ms/step - loss: 2.1497 - acc: 0.2485 - val_loss: 1.8979 - val_acc: 0.3624
Epoch 2/25
94/94 [==============================] - 5s 49ms/step - loss: 1.6330 - acc: 0.4648 - val_loss: 1.3772 - val_acc: 0.5950
Epoch 3/25
94/94 [==============================] - 5s 48ms/step - loss: 1.2279 - acc: 0.6295 - val_loss: 1.0835 - val_acc: 0.6795
Epoch 4/25
94/94 [==============================] - 4s 47ms/step - loss: 1.0226 - acc: 0.6914 - val_loss: 0.9252 - val_acc: 0.7232
Epoch 5/25
94/94 [==============================] - 4s 47ms/step - loss: 0.8952 - acc: 0.7288 - val_loss: 0.8316 - val_acc: 0.7501
Epoch 6/25
94/94 [==============================] - 4s 46ms/step - loss: 0.8089 - acc: 0.7559 - val_loss: 0.7370 - val_acc: 0.7786
Epoch 7/25
94/94 [==============================] - 4s 47ms/step - loss: 0.7348 - acc: 0.7785 - val_loss: 0.6799 - val_acc: 0.7974
Epoch 8/25
94/94 [==============================] - 4s 47ms/step - loss: 0.6806 - a

In [11]:
test_loss, test_acc = model.evaluate(x=x_test_eager.numpy(), 
                                     y=y_test_eager.numpy())

print("test_accracy: ", test_acc)

313/313 [==============================] - 6s 18ms/step
test_accracy:  0.8839
